# Data exploration

## I.Import relevant librairires

In [ ]:
import os
import sys
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Below, you can import the different modules coded in ``WeakSegNet/src``

In [ ]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..")) # This is the repo directory: WeakSegNet/
SRC_PATH = os.path.join(BASE_DIR, "src")
sys.path.append(SRC_PATH)

from dataset import data_loading, inverse_normalize

## II.Load the data

In [ ]:
# Sanity check: are the trainval.txt and test.txt files available

FILE_PATH = os.path.join(BASE_DIR, "data")

split = "trainval" # or "test"
print(f"Looking for {split}.txt in:", os.path.join(FILE_PATH, "annotations", f"{split}.txt"))
print("Does the file exists?", os.path.exists(os.path.join(FILE_PATH, "annotations", "test.txt")))

## III.Data visualization

In [ ]:
# Data splits: configure the size of the splits
batch_size_train, batch_size_val, batch_size_test = 32, 16, 32
val_split = 0.2
size = (batch_size_train, batch_size_val, batch_size_test, val_split)

In [ ]:
# Configure the resizing of the images
image_size = 256

### 1.Fully supervised case

In [ ]:
train_loader, val_loader, test_loader = data_loading(path=FILE_PATH, data_split_size=size, image_size=image_size)

In [ ]:
image_batch, mask_batch, info_batch = next(iter(train_loader))

In [ ]:
id = 8
image = inverse_normalize(image_batch[id]).permute(1, 2, 0).cpu().numpy()
mask = mask_batch[id].squeeze().cpu().numpy()
info = info_batch["name"][id]

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image: " + info)
plt.axis('off')
plt.grid(False)

plt.subplot(1, 2, 2)
plt.imshow(mask, cmap="grey")
plt.title(f"GT mask")
plt.axis('off')

plt.grid(False)
plt.show()

In [ ]:
unique_vals = torch.unique(mask_batch[0])
print("Unique mask values:", unique_vals)

### 2.Weakly supervised case: bounding boxes

In [ ]:
box = info_batch["bbox"][0] # TODO: find a way to scale the bouding box to the image size after resizing

In [ ]:
box

In [ ]:
plt.imshow(image)
plt.title("Image")
plt.axis('off')
plt.grid(False)
ax = plt.gca()
width = box[1] - box[0]
heigth = box[3] - box[2]
ax.add_patch(patches.Rectangle((box[0], box[2]), width, heigth, linewidth=2, edgecolor='r', facecolor='none'))
plt.show()

### 3- Weakly supervised case: CMAP